<a href="https://colab.research.google.com/github/UrangaEsteban/progcon/blob/procesos_java/tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://repo1.maven.org/maven2/com/google/code/gson/gson/2.10.1/gson-2.10.1.jar --output "gson-2.10.1.jar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0  3502k      0 --:--:-- --:--:-- --:--:-- 3502k


In [ ]:
!ls -lash

total 296K
4.0K drwxr-xr-x 1 root root 4.0K Apr 14 03:38 .
4.0K drwxr-xr-x 1 root root 4.0K Apr 14 03:36 ..
4.0K drwxr-xr-x 4 root root 4.0K Apr 12 13:33 .config
280K -rw-r--r-- 1 root root 277K Apr 14 03:38 gson-2.10.1.jar
4.0K drwxr-xr-x 1 root root 4.0K Apr 12 13:34 sample_data


In [ ]:
!echo '{"A":["B","C","D"],"B":["E","F"],"D":["G"]}' > target.json

In [ ]:
%%writefile ProcessTree.java
import com.google.gson.Gson;
import com.google.gson.reflect.TypeToken;
import java.lang.reflect.Type;
import java.io.StringReader;
import java.io.PrintWriter;
import java.io.FileOutputStream;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.FileSystems;
import java.nio.file.FileSystem;

import java.util.*;

public class ProcessTree 
{
  public static void main(String[] args) 
  {
    String self = (args.length == 0) ? "A" : String.valueOf(args[0]);
    
    info(self);
    waitForChildren(spawnChildren(getDataMap(), self));

    try 
    {
      Thread.sleep(5000);
    } catch (InterruptedException e) 
    {
      System.err.println("Se ha producido una interrupción");
    }

  }

  public static void waitForChildren(List<Process> children) 
  {
    for(Process child : children) 
    {
      try 
      {
        child.waitFor();
      } catch (InterruptedException e) 
      {
        System.err.println("Se ha producido una interrupción");
      }
    }
  }

  public static HashMap<String, ArrayList<String>> getDataMap() 
  {
    Gson gson = new Gson();
    Type customType = new TypeToken<HashMap<String, ArrayList<String>>>() {}.getType();
    return gson.fromJson(getFileData(DATA_FILE_NAME), customType); 
  }

  public static String getFileData(String fileName) 
  {
    try 
    {
      return String.join("", 
        Files.readAllLines(FileSystems.getDefault().getPath(fileName))
      );
    } catch (NullPointerException e) 
    {
      e.printStackTrace();
    } catch (IOException e) 
    {
      System.err.println("No se pudo leer el archivo " + fileName);
      e.printStackTrace();
    }
    return "";
  }

  public synchronized static void info(String selfData) 
  {
    try ( PrintWriter out = new PrintWriter(new FileOutputStream(OUT, true), true); ) 
    {
      out.print("Soy el proceso " + selfData + ". ");
      out.print("Mi PID es: " + ProcessHandle.current().pid() + ". ");
      out.print("El PID de mi padre es: " + ProcessHandle.current().parent().get().pid() + ".\n");
      out.flush();
    } catch (IOException e) 
    {
      System.err.println("No se pudo abrir el archivo: " + OUT);
      e.printStackTrace();
    } catch (NoSuchElementException e) 
    {
      System.err.println("No tengo padre: " + OUT);
      e.printStackTrace();
    }
  }

  public static ArrayList<Process> spawnChildren(HashMap<String, ArrayList<String>> dataMap, String self) 
  {
    ArrayList<Process> children = new ArrayList<>();
    if (dataMap.keySet().contains(self)) 
    {
      for(String childValue : dataMap.get(self)) 
      {
        try 
        {
          children.add(new ProcessBuilder("java", "-cp", ".:gson-2.10.1.jar:gson-2.10.1.jar:", "ProcessTree", childValue, "1>salidaJava", "2>errors").start());
        } catch (IOException e) 
        {
          System.err.println("No se pudieron crear mas procesos:  " + childValue);
        }
      }
    }
    return children;
  }

  final static String DATA_FILE_NAME = "target.json";
  final static String OUT = "miSalidaJava";
}

Overwriting ProcessTree.java


In [ ]:
!javac -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessTree.java

In [ ]:
!nohup java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessTree 1>salidaJava 2>errors &  

En la siguiente celda de código, lo que se hace es mostrar el árbol de procesos sin tener que copiar a manopla el process ID después de ejecutar un ps. Para reutilizarlo, hay que cambiar el /ProcessTree/ por una expresión regular que represente el nombre que se le da al proceso cuando se ejecuta un PS, y si el proceso padre no es el 1 hay que remover la parte del "&& $2 == 1 {print $1}" que valida que el parent process id sea 1

In [ ]:
!pstree -pcT "$(eval ps -eo pid,ppid,args | awk '$NF ~ /ProcessTree/ && $2 == 1 {print $1}')"

java(1684)─┬─java(1701)─┬─java(1774)
           │            └─java(1778)
           ├─java(1703)
           └─java(1706)───java(1759)


In [ ]:
!cat miSalidaJava

Soy el proceso A. Mi PID es: 1953. El PID de mi padre es: 1.
Soy el proceso B. Mi PID es: 1970. El PID de mi padre es: 1953.
Soy el proceso C. Mi PID es: 1977. El PID de mi padre es: 1953.
Soy el proceso D. Mi PID es: 1980. El PID de mi padre es: 1953.
Soy el proceso F. Mi PID es: 2033. El PID de mi padre es: 1970.
Soy el proceso E. Mi PID es: 2028. El PID de mi padre es: 1970.
Soy el proceso G. Mi PID es: 2061. El PID de mi padre es: 1980.


In [ ]:
!rm miSalidaJava

In [ ]:
!cat errors

In [ ]:
!ps -ef 

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 03:36 ?        00:00:00 /sbin/docker-init -- /datala
root           7       1  0 03:36 ?        00:00:01 /tools/node/bin/node /datala
root          19       0  0 03:36 ?        00:00:00 tail -n +0 -F /root/.config/
root          30       7  0 03:36 ?        00:00:00 /bin/bash -e /usr/local/cola
root          32       1  0 03:36 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          34      32  0 03:36 ?        00:00:00 /usr/colab/bin/kernel_manage
root          60       7  2 03:36 ?        00:00:06 [python3] <defunct>
root          61       7  0 03:36 ?        00:00:00 python3 /usr/local/bin/colab
root          82       7  1 03:37 ?        00:00:03 /usr/bin/python3 /usr/local/
root          83       7  0 03:37 ?        00:00:00 /usr/local/bin/dap_multiplex
root         291      82  1 03:37 ?        00:00:03 /usr/bin/python3 -m ipykerne
root         322       1  0 03:37 ?        00:00:00 /usr/bin/p